# Quantum Variational Simulator

## The time-dependent variational principle

This principle corresponding to the Schrödinger equation reads $\delta \int_{t_{i}}^{t_{f}} d t L=0$, where the Lagrangian is $L=\langle\Psi(t)|\left(i \frac{\partial}{\partial t}-H\right)| \Psi(t)\rangle$. Assuming that the state $| \Psi(t)\rangle$ is determined by a set of real parameters $\left\{\lambda_{k}(t)\right\}$, that is $|\Psi(t)\rangle \equiv\left|\Psi\left(\lambda_{1}, \lambda_{2}, \ldots\right)\right\rangle$. The the Lagrangian can be rewritten as 

$$
L=i \sum_{k}\langle\Psi| \frac{\partial|\Psi\rangle}{\partial \lambda_{k}} \dot{\lambda}_{k}-\langle\Psi|H| \Psi\rangle
$$

which is a function of parameters $\{\lambda_{k}\}$ and their time derivatives $\left\{\dot{\lambda}_{k}=\left[d \lambda_{k} / d t\right]\right\}$. After applying the Euler-Lagrange equation, the evolution of parameters is

$$
\sum_{q} A_{k, q} \dot{\lambda}_{q}=V_{k}
$$

where

$$
A_{k, q}=i  \frac{\partial\langle\Psi|}{\partial \lambda_{k}} \frac{\partial|\Psi\rangle}{\partial \lambda_{q}}+\text { H.c. } \quad \text{and} \quad V_{k}= \frac{\partial\langle\Psi|}{\partial \lambda_{k}} H|\Psi\rangle+\text { H.c. }
$$

## Variational algorithm on a hybrid computer